In [28]:
import pandas as pd

In [29]:
credits = pd.read_csv('../data/credits.csv')
movies = pd.read_csv('../data/movies.csv')

In [30]:
print("movies shape: " + str(movies.shape))
print("credits shape: " + str(credits.shape))

movies shape: (4803, 20)
credits shape: (4803, 4)


In [31]:
merged_df = movies.merge(credits, on='title')

In [32]:
merged_df.shape

(4809, 23)

In [33]:
merged_df = merged_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
merged_df.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [34]:
merged_df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [35]:
merged_df.dropna(inplace=True)
merged_df.duplicated().sum()

np.int64(0)

In [36]:
merged_df.iloc[0]['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [37]:
import ast 

def convert(text):
    converted_list = []
    for i in ast.literal_eval(text):
        converted_list.append(i['name']) 
    return converted_list

merged_df['genres'] = merged_df['genres'].apply(convert)
merged_df['keywords'] = merged_df['keywords'].apply(convert)
merged_df['cast'] = merged_df['cast'].apply(convert)

In [38]:
def get_director(text):
    director_list = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            director_list.append(i['name'])
            break
    return director_list

merged_df['crew'] = merged_df['crew'].apply(get_director)

In [39]:
merged_df['overview'] = merged_df['overview'].apply(lambda x: x.split())
merged_df['tags'] = merged_df['overview'] + merged_df['genres'] + merged_df['keywords'] + merged_df['cast'] + merged_df['crew']

In [40]:
merged_df.head(4)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan],"[Following, the, death, of, District, Attorney..."


In [41]:
new_df = merged_df[['movie_id', 'title', 'tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

new_df.head(2)

C:\Users\Thiago Barros\AppData\Local\Temp\ipykernel_27820\1840262508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
C:\Users\Thiago Barros\AppData\Local\Temp\ipykernel_27820\1840262508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [42]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [44]:
def stems(text):
    return ' '.join([ps.stem(word) for word in text.split()])
new_df['tags'] = new_df['tags'].apply(stems)

new_df.iloc[0]['tags']

C:\Users\Thiago Barros\AppData\Local\Temp\ipykernel_27820\2057118448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stems)


'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi scienc fiction cultur clash futur space war space coloni societi space travel futurist romanc space alien tribe alien planet cgi marin soldier battl love affair anti war power relat mind and soul 3d sam worthington zoe saldana sigourney weaver stephen lang michel rodriguez giovanni ribisi joel david moor cch pounder we studi laz alonso dileep rao matt gerald sean anthoni moran jason whyte scott lawrenc kelli kilgour jame patrick pitt sean patrick murphi peter dillon kevin dorman kelson henderson david van horn jacob tomuri michael blain-rozgay jon curri luke hawker woodi schultz peter mensah sonia yee jahnel curfman ilram choi kyla warren lisa roumain debra wilson chri mala taylor kibbi jodi landau juli lamm cullen b. madden joseph bradi madden franki torr austin wilson sara wilson tamica washington-mil

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

vectors = cv.fit_transform(new_df['tags']).toarray()
vectors.shape

(4806, 5000)

In [46]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

similarity.shape

(4806, 4806)

In [52]:
similarity[:5]

array([[1.        , 0.10881351, 0.07927124, ..., 0.03464015, 0.01900543,
        0.01536191],
       [0.10881351, 1.        , 0.07938842, ..., 0.0375823 , 0.01374643,
        0.02222222],
       [0.07927124, 0.07938842, 1.        , ..., 0.0238688 , 0.05238277,
        0.        ],
       [0.22212349, 0.13529234, 0.17722065, ..., 0.03432103, 0.09415174,
        0.05073463],
       [0.22177267, 0.06604949, 0.08090071, ..., 0.02553215, 0.05603318,
        0.02264554]])

In [47]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    return movies_list[]

In [51]:
recommend('Spider-Man 2')

Spider-Man 3
Spider-Man
The Amazing Spider-Man 2
The Amazing Spider-Man
X-Men Origins: Wolverine


'X-Men Origins: Wolverine'

In [54]:
import pickle 

pickle.dump(new_df, open('../artifacts/movies_list.pkl', 'wb'))
pickle.dump(similarity, open('../artifacts/similarity.pkl', 'wb'))